# Detecting Building defects using CNN

### Importing Neccessary Libraries

In [1]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


### Image Data Agumentation

In [2]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [5]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(
    r'E:\GRIP Venkatesh\Rock_Classification\Rock_Classification_Dataset\train_set',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(
    r'E:\GRIP Venkatesh\Rock_Classification\Rock_Classification_Dataset\test_set',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='categorical') 

Found 700 images belonging to 5 classes.
Found 300 images belonging to 5 classes.


In [9]:
print(x_train.class_indices)#checking the number of classes

{'blue calcite': 0, 'limestone': 1, 'marble': 2, 'olivine': 3, 'red crystal': 4}


In [10]:
print(x_test.class_indices)#checking the number of classes

{'blue calcite': 0, 'limestone': 1, 'marble': 2, 'olivine': 3, 'red crystal': 4}


In [11]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 140, 1: 140, 2: 140, 3: 140, 4: 140})

### Creating the model

In [16]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=5, activation='softmax')) # softmax for more than 2



In [17]:
classifier.summary()#summary of our model

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_3 (Dense)              (None, 5)                

### Compiling the model

In [18]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [19]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=50, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

Epoch 1/50
140/140 [==============================] - 11s 76ms/step - loss: 1.1382 - acc: 0.4829 - val_loss: 0.9346 - val_acc: 0.5667
Epoch 2/50
140/140 [==============================] - 4s 29ms/step - loss: 0.6776 - acc: 0.7286 - val_loss: 0.4576 - val_acc: 0.8467
Epoch 3/50
140/140 [==============================] - 4s 29ms/step - loss: 0.4876 - acc: 0.7971 - val_loss: 1.0653 - val_acc: 0.6933
Epoch 4/50
140/140 [==============================] - 4s 29ms/step - loss: 0.4732 - acc: 0.8171 - val_loss: 0.4588 - val_acc: 0.8467
Epoch 5/50
140/140 [==============================] - 4s 29ms/step - loss: 0.3873 - acc: 0.8329 - val_loss: 0.3770 - val_acc: 0.8700
Epoch 6/50
140/140 [==============================] - 4s 29ms/step - loss: 0.3618 - acc: 0.8543 - val_loss: 0.5339 - val_acc: 0.8800
Epoch 7/50
140/140 [==============================] - 4s 29ms/step - loss: 0.3561 - acc: 0.8571 - val_loss: 0.3267 - val_acc: 0.8833
Epoch 8/50
140/140 [==============================] - 4s 29ms/step -

### Saving our model

In [20]:
# Save the model
classifier.save('rock.h5')

In [ ]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

### Predicting our results

In [2]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("rock.h5") #loading the model for testing

In [3]:
img = image.load_img(r"E:\GRIP Venkatesh\Rock_Classification\Rock_Classification_Dataset\o1.jpg",
                     grayscale=False,target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict_classes(x)#predicting the classes
pred

C:\Users\smartbridge\Anaconda3\envs\rock\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([1], dtype=int64)

In [4]:
index=['blue calcite', 'limestone', 'marble', 'olivine', 'red crystal']
result=str(index[pred[0]])
result

'limestone'

In [2]:
!pip install jupyterthemes

  Using cached https://files.pythonhosted.org/packages/8a/08/9dee6dfd7f2aad6c30282d55c8f495b4dc1e4747b4e2bdbeb80572ddf312/jupyterthemes-0.20.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f8/d2/665cda6614e3556eaeb7553a3a2963624c2e3bc9636777a1bb654b87b027/lesscpy-0.14.0-py2.py3-none-any.whl


In [3]:
!jt -t  monokai

In [ ]:
!jt -t 